## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Nanortalik,GL,60.1432,-45.2371,39.34,81,95,3.96,overcast clouds
1,1,Kavaratti,IN,10.5669,72.6420,82.81,76,5,7.74,clear sky
2,2,Mataura,NZ,-46.1927,168.8643,39.36,99,35,1.97,scattered clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,61.18,96,100,1.01,overcast clouds
4,4,Raudeberg,NO,61.9875,5.1352,38.07,77,23,8.66,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Kavaratti,IN,10.5669,72.6420,82.81,76,5,7.74,clear sky
7,7,Manuk Mangkaw,PH,4.8000,119.8500,81.86,79,54,9.73,broken clouds
8,8,Hilo,US,19.7297,-155.0900,80.33,70,20,8.05,few clouds
9,9,Sechura,PE,-5.5569,-80.8222,82.99,57,82,10.47,broken clouds
11,11,Kilindoni,TZ,-7.9139,39.6668,81.57,75,12,7.49,few clouds
13,13,Kalmunai,LK,7.4167,81.8167,78.22,84,55,8.21,broken clouds
14,14,Sabha,LY,27.0377,14.4283,77.41,13,100,5.06,overcast clouds
16,16,Providencia,MX,28.7167,-111.5833,81.21,37,0,11.99,clear sky
21,21,Porto Novo,BJ,6.4965,2.6036,82.20,78,100,2.26,light rain
24,24,Atuona,PF,-9.8000,-139.0333,78.53,76,12,7.67,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                209
City                   209
Country                209
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Weather Description    209
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Kavaratti,IN,82.81,clear sky,10.5669,72.6420,
7,Manuk Mangkaw,PH,81.86,broken clouds,4.8000,119.8500,
8,Hilo,US,80.33,few clouds,19.7297,-155.0900,
9,Sechura,PE,82.99,broken clouds,-5.5569,-80.8222,
11,Kilindoni,TZ,81.57,few clouds,-7.9139,39.6668,
13,Kalmunai,LK,78.22,broken clouds,7.4167,81.8167,
14,Sabha,LY,77.41,overcast clouds,27.0377,14.4283,
16,Providencia,MX,81.21,clear sky,28.7167,-111.5833,
21,Porto Novo,BJ,82.20,light rain,6.4965,2.6036,
24,Atuona,PF,78.53,few clouds,-9.8000,-139.0333,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Kavaratti,IN,82.81,clear sky,10.5669,72.6420,Hotels in Lakshadweep Islands
8,Hilo,US,80.33,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
9,Sechura,PE,82.99,broken clouds,-5.5569,-80.8222,Hospedaje Costa Bella
11,Kilindoni,TZ,81.57,few clouds,-7.9139,39.6668,Bustani B&B
13,Kalmunai,LK,78.22,broken clouds,7.4167,81.8167,VS Villa
14,Sabha,LY,77.41,overcast clouds,27.0377,14.4283,Foundation Taaj Asala for Historical Research ...
21,Porto Novo,BJ,82.20,light rain,6.4965,2.6036,Tour Eiffel Hotel Benin
24,Atuona,PF,78.53,few clouds,-9.8000,-139.0333,Villa Enata
26,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
36,Vanimo,PG,78.80,moderate rain,-2.6741,141.3028,Vahmoneh Lodge Vanimo


In [39]:

# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))